In [65]:
#모델에 쓰인 파일들은 따로 첨부했습니다. 모델 재현시 경로설정 부탁드립니다.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
import math
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import sklearn
from sklearn.utils import shuffle


In [67]:
#버전확인
print('python', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('matplotlib', matplotlib.__version__)
print('keras', keras.__version__)
print('sklearn', sklearn.__version__)

python 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]
pandas 1.0.5
numpy 1.18.5
matplotlib 3.2.2
keras 2.4.3
sklearn 0.22.2.post1


In [68]:
seed=0
np.random.seed(seed)
tf.random.set_seed(3)

**모델 만들기**

In [ ]:
#데이터 프레임임을 시계열로 넣을 수 있게 데이터 정제
def create_dataset(dataset, window_size):
  data_X, data_Y=[], []
  for i in range(len(dataset)-window_size-window_size-1):
    a= dataset[i:(i+window_size), 0]
    data_X.append(a)
    data_Y.append(dataset[i+window_size+window_size, 0])
  return(np.array(data_X), np.array(data_Y))

def fit_model(train_X, train_Y, window_size):
  model = Sequential()
    
  model.add(LSTM(4, 
                   input_shape=(1, window_size)))
  model.add(Dense(10))
  model.add(Dense(1))
  model.compile(loss = "mean_squared_error", optimizer = "adam")
  model.fit(train_X, train_Y, epochs = 50, batch_size = 1, verbose = 2)
  return(model)

**타자 모델링**

In [ ]:
#연도별로 타자의 데이터 불러오기
YEAR=[2016, 2017, 2018, 2019]
#(/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/)까지 경로를 수정하셔야 합니다.
for i in YEAR:  
  s1="""data_%d=pd.read_csv('/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_%d.csv')
data_%d=data_%d[['G_ID', 'T_ID', 'AB', 'HIT']]
"""%(i, i, i, i)  
  exec(s1)

In [ ]:
#연도별 데이터를 정제해 경기 진행에 따른 누적타율을 구할 수 있는 모델을 만듬
LAST_AVG_2016=[]
LAST_AVG_2017=[]
LAST_AVG_2018=[]
LAST_AVG_2019=[]

YEAR=[2016, 2017, 2018, 2019]
TEAM_LIST = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']
for i in YEAR:
  for j in TEAM_LIST:
    s1 = "%s_AVG_%d = pd.DataFrame(data_%d[data_%d['T_ID']==j])"%(j, i, i, i)
    exec(s1)

for i in YEAR:
  for j in TEAM_LIST:
    s2 = "%s_AVG_%d['누적타석'] = %s_AVG_%d['AB'].cumsum()"%(j,i, j, i)
    exec(s2)

for i in YEAR:
  for j in TEAM_LIST:
    s3 = "%s_AVG_%d['누적안타'] = %s_AVG_%d['HIT'].cumsum()"%(j,i,j,i)
    exec(s3)

for i in YEAR:
  for j in TEAM_LIST:
    s4 = "%s_AVG_%d['누적타율'] = %s_AVG_%d['누적안타']/%s_AVG_%d['누적타석']"%(j,i, j,i, j,i)
    exec(s4)

for i in YEAR:
  for j in TEAM_LIST:
    s5 = "%s_AVG_%d.reset_index(drop=True, inplace=True)"%(j,i)
    exec(s5)

for i in YEAR:
  for j in TEAM_LIST:
    s11="LAST_AVG_%d.append(%s_AVG_%d['누적타율'][143])"%(i, j,i)
    exec(s11)

In [ ]:
#모델을 돌려 연도별 144경기 예측값 도출
TEAM_AVG = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']
PRED_AVG=[]
PRED_AVG_2016=[]
PRED_AVG_2017=[]
PRED_AVG_2018=[]
PRED_AVG_2019=[]
YEAR=[2016, 2017, 2018, 2019]

for i in YEAR:
  PRED_AVG=[]
  for j in TEAM_AVG:
    s10 = "data=%s_AVG_%d[['누적타율']]"%(j, i)
    exec(s10)
    data_raw = data.values.astype("float32")
    data_raw=np.array(data_raw)
    data_raw=data_raw.reshape(-1,1)
    scaler = MinMaxScaler(feature_range = (0, 1))
    dataset = scaler.fit_transform(data_raw)

    train=dataset

    window_size = 24
    train_X, train_Y = create_dataset(train, window_size)

    train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))

    # Fit the first model.
    model1 = fit_model(train_X, train_Y, window_size)

    pred=scaler.inverse_transform(model1.predict(train_X))

    PRED_AVG.append(pred[-1][0])

  s12 = "PRED_AVG_%d=PRED_AVG"%(i)
  exec(s12)

Epoch 1/50
95/95 - 0s - loss: 0.0603
Epoch 2/50
95/95 - 0s - loss: 0.0020
Epoch 3/50
95/95 - 0s - loss: 0.0013
Epoch 4/50
95/95 - 0s - loss: 9.4166e-04
Epoch 5/50
95/95 - 0s - loss: 6.8873e-04
Epoch 6/50
95/95 - 0s - loss: 6.5435e-04
Epoch 7/50
95/95 - 0s - loss: 5.9109e-04
Epoch 8/50
95/95 - 0s - loss: 6.0298e-04
Epoch 9/50
95/95 - 0s - loss: 6.0729e-04
Epoch 10/50
95/95 - 0s - loss: 5.9175e-04
Epoch 11/50
95/95 - 0s - loss: 5.5362e-04
Epoch 12/50
95/95 - 0s - loss: 7.0299e-04
Epoch 13/50
95/95 - 0s - loss: 6.3507e-04
Epoch 14/50
95/95 - 0s - loss: 5.4604e-04
Epoch 15/50
95/95 - 0s - loss: 5.5009e-04
Epoch 16/50
95/95 - 0s - loss: 5.2506e-04
Epoch 17/50
95/95 - 0s - loss: 5.2627e-04
Epoch 18/50
95/95 - 0s - loss: 5.5911e-04
Epoch 19/50
95/95 - 0s - loss: 5.3514e-04
Epoch 20/50
95/95 - 0s - loss: 5.0612e-04
Epoch 21/50
95/95 - 0s - loss: 7.5554e-04
Epoch 22/50
95/95 - 0s - loss: 5.0271e-04
Epoch 23/50
95/95 - 0s - loss: 5.5276e-04
Epoch 24/50
95/95 - 0s - loss: 5.6336e-04
Epoch 25/50
9

In [ ]:
  #최종 타율 예측값 산출
FINAL_2016=pd.DataFrame(data=[LAST_AVG_2016,PRED_AVG_2016], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_2016=FINAL_2016.T
FINAL_2017=pd.DataFrame(data=[LAST_AVG_2017,PRED_AVG_2017], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_2017=FINAL_2017.T
FINAL_2018=pd.DataFrame(data=[LAST_AVG_2018,PRED_AVG_2018], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_2018=FINAL_2018.T
FINAL_2019=pd.DataFrame(data=[LAST_AVG_2019,PRED_AVG_2019], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_2019=FINAL_2019.T

In [ ]:
#120경기 기존 값 도출
HALF_AVG_2016=[]
HALF_AVG_2017=[]
HALF_AVG_2018=[]
HALF_AVG_2019=[]

for i in YEAR:
  for j in TEAM_LIST:
    s11="HALF_AVG_%d.append(%s_AVG_%d['누적타율'][119])"%(i, j,i)
    exec(s11)

In [ ]:
#역추정하는 식
YEAR=[2016, 2017, 2018, 2019]
for i in YEAR:  
  s1="""FINAL_AVG_%d=pd.DataFrame(data=[LAST_AVG_%d,HALF_AVG_%d, PRED_AVG_%d], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_AVG_%d=FINAL_AVG_%d.T
FINAL_AVG_%d.columns=['최종타율', '120경기 타율', '예측최종타율']
FINAL_AVG_%d['잔여경기타율']=(FINAL_AVG_%d['최종타율']-(120/144)*FINAL_AVG_%d['120경기 타율'])*(144/24)
FINAL_AVG_%d['예측잔여경기타율']=(FINAL_AVG_%d['예측최종타율']-(120/144)*FINAL_AVG_%d['120경기 타율'])*(144/24)
"""%(i, i, i, i, i, i, i, i, i, i, i, i, i)
  exec(s1)


In [ ]:
FINAL_AVG=pd.concat([FINAL_AVG_2016, FINAL_AVG_2017, FINAL_AVG_2018, FINAL_AVG_2019])

In [ ]:
FINAL_AVG

,최종타율,120경기 타율,예측최종타율,잔여경기타율,예측잔여경기타율
LG,0.289844,0.288648,0.291801,0.295823,0.307570
SS,0.292951,0.294769,0.290300,0.283861,0.267957
HH,0.289293,0.289437,0.288862,0.288577,0.285988
LT,0.287742,0.283277,0.286848,0.310068,0.304700
NC,0.290993,0.291697,0.290662,0.287473,0.285488
HT,0.285857,0.291232,0.288998,0.258982,0.277826
WO,0.293387,0.294089,0.293652,0.289874,0.291467
SK,0.291459,0.288628,0.291645,0.305618,0.306730
KT,0.275841,0.271868,0.273358,0.295706,0.280810
OB,0.298176,0.298166,0.297825,0.298225,0.296118


**투수 모델링**

In [ ]:
#연도별로 투수의 데이터 불러오기
YEAR=[2016, 2017, 2018, 2019]
#(/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/)까지 경로를 수정하셔야 합니다.
for i in YEAR:  
  s1="""pitcher_%d=pd.read_csv('/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_%d.csv')
pitcher_%d=pitcher_%d[['G_ID', 'T_ID', 'INN2', 'ER']]
"""%(i, i, i, i)
  exec(s1)


In [ ]:
#연도별 데이터를 정제해 경기 진행에 따른 누적방어율을 구할 수 있는 모델을 만듬
def create_dataset(dataset, window_size):
  data_X, data_Y=[], []
  for i in range(len(dataset)-window_size-window_size-1):
    a= dataset[i:(i+window_size), 0]
    data_X.append(a)
    data_Y.append(dataset[i+window_size+window_size, 0])
  return(np.array(data_X), np.array(data_Y))

def fit_model(train_X, train_Y, window_size):
  model = Sequential()
  model.add(LSTM(4, 
                   input_shape=(1, window_size)))
  model.add(Dense(10))
  model.add(Dense(1))
  model.compile(loss = "mean_squared_error", optimizer = "adam")
  model.fit(train_X, train_Y, epochs = 50, batch_size = 1, verbose = 2)
  return(model)

In [ ]:
#모델을 돌려 연도별 144경기 예측값 도출
LAST_ERA_2016=[]
LAST_ERA_2017=[]
LAST_ERA_2018=[]
LAST_ERA_2019=[]

YEAR=[2016, 2017, 2018, 2019]
TEAM_LIST = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']
for i in YEAR:
  for j in TEAM_LIST:
    s1 = "%s_ERA_%d = pd.DataFrame(pitcher_%d[pitcher_%d['T_ID']==j])"%(j, i, i, i)
    exec(s1)

for i in YEAR:
  for j in TEAM_LIST:
    s2 = "%s_ERA_%d['누적이닝'] = %s_ERA_%d['INN2'].cumsum()"%(j,i, j, i)
    exec(s2)

for i in YEAR:
  for j in TEAM_LIST:
    s3 = "%s_ERA_%d['누적자책'] = %s_ERA_%d['ER'].cumsum()"%(j,i,j,i)
    exec(s3)

for i in YEAR:
  for j in TEAM_LIST:
    s4 = "%s_ERA_%d['누적방어율'] = %s_ERA_%d['누적자책']*27/%s_ERA_%d['누적이닝']"%(j,i, j,i, j,i)
    exec(s4)

for i in YEAR:
  for j in TEAM_LIST:
    s5 = "%s_ERA_%d.reset_index(drop=True, inplace=True)"%(j,i)
    exec(s5)

for i in YEAR:
  for j in TEAM_LIST:
    s11="LAST_ERA_%d.append(%s_ERA_%d['누적방어율'][143])"%(i, j,i)
    exec(s11)

In [ ]:
#최종 방어율 예측값 산출
TEAM_ERA = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']
PRED_ERA=[]
PRED_ERA_2016=[]
PRED_ERA_2017=[]
PRED_ERA_2018=[]
PRED_ERA_2019=[]
YEAR=[2016, 2017, 2018, 2019]

for i in YEAR:
  PRED_ERA=[]
  for j in TEAM_ERA:
    s10 = "data=%s_ERA_%d[['누적방어율']]"%(j, i)
    exec(s10)
    data_raw = data.values.astype("float32")
    data_raw=np.array(data_raw)
    data_raw=data_raw.reshape(-1,1)
    scaler = MinMaxScaler(feature_range = (0, 1))
    dataset = scaler.fit_transform(data_raw)

    train=dataset

    window_size = 24
    train_X, train_Y = create_dataset(train, window_size)

    train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))

    # Fit the first model.
    model1 = fit_model(train_X, train_Y, window_size)

    pred=scaler.inverse_transform(model1.predict(train_X))

    PRED_ERA.append(pred[-1][0])

  s12 = "PRED_ERA_%d=PRED_ERA"%(i)
  exec(s12)

Epoch 1/50
95/95 - 0s - loss: 0.0947
Epoch 2/50
95/95 - 0s - loss: 0.0035
Epoch 3/50
95/95 - 0s - loss: 0.0032
Epoch 4/50
95/95 - 0s - loss: 0.0032
Epoch 5/50
95/95 - 0s - loss: 0.0030
Epoch 6/50
95/95 - 0s - loss: 0.0030
Epoch 7/50
95/95 - 0s - loss: 0.0026
Epoch 8/50
95/95 - 0s - loss: 0.0026
Epoch 9/50
95/95 - 0s - loss: 0.0025
Epoch 10/50
95/95 - 0s - loss: 0.0024
Epoch 11/50
95/95 - 0s - loss: 0.0021
Epoch 12/50
95/95 - 0s - loss: 0.0024
Epoch 13/50
95/95 - 0s - loss: 0.0022
Epoch 14/50
95/95 - 0s - loss: 0.0021
Epoch 15/50
95/95 - 0s - loss: 0.0020
Epoch 16/50
95/95 - 0s - loss: 0.0022
Epoch 17/50
95/95 - 0s - loss: 0.0024
Epoch 18/50
95/95 - 0s - loss: 0.0023
Epoch 19/50
95/95 - 0s - loss: 0.0019
Epoch 20/50
95/95 - 0s - loss: 0.0019
Epoch 21/50
95/95 - 0s - loss: 0.0022
Epoch 22/50
95/95 - 0s - loss: 0.0017
Epoch 23/50
95/95 - 0s - loss: 0.0019
Epoch 24/50
95/95 - 0s - loss: 0.0018
Epoch 25/50
95/95 - 0s - loss: 0.0018
Epoch 26/50
95/95 - 0s - loss: 0.0019
Epoch 27/50
95/95 - 0

In [ ]:
#최종 방어율 예측값 산출
FINAL_ERA_2016=pd.DataFrame(data=[LAST_ERA_2016,PRED_ERA_2016], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_ERA_2016=FINAL_ERA_2016.T
FINAL_ERA_2017=pd.DataFrame(data=[LAST_ERA_2017,PRED_ERA_2017], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_ERA_2017=FINAL_ERA_2017.T
FINAL_ERA_2018=pd.DataFrame(data=[LAST_ERA_2018,PRED_ERA_2018], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_ERA_2018=FINAL_ERA_2018.T
FINAL_ERA_2019=pd.DataFrame(data=[LAST_ERA_2019,PRED_ERA_2019], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_ERA_2019=FINAL_ERA_2019.T

In [ ]:
#120경기 기존 값 도출
HALF_ERA_2016=[]
HALF_ERA_2017=[]
HALF_ERA_2018=[]
HALF_ERA_2019=[]

for i in YEAR:
  for j in TEAM_LIST:
    s11="HALF_ERA_%d.append(%s_ERA_%d['누적방어율'][119])"%(i, j,i)
    exec(s11)

In [ ]:
#역추정하는 식
YEAR=[2016, 2017, 2018, 2019]
for i in YEAR:  
  s1="""FINAL_ERA_%d=pd.DataFrame(data=[LAST_ERA_%d,HALF_ERA_%d, PRED_ERA_%d], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_ERA_%d=FINAL_ERA_%d.T
FINAL_ERA_%d.columns=['최종방어율', '120경기 방어율', '예측최종방어율']
FINAL_ERA_%d['잔여경기방어율']=(FINAL_ERA_%d['최종방어율']-(120/144)*FINAL_ERA_%d['120경기 방어율'])*(144/24)
FINAL_ERA_%d['예측잔여경기방어율']=(FINAL_ERA_%d['예측최종방어율']-(120/144)*FINAL_ERA_%d['120경기 방어율'])*(144/24)
"""%(i, i, i, i, i, i, i, i, i, i, i, i, i)
  exec(s1)


In [ ]:
FINAL_ERA=pd.concat([FINAL_ERA_2016, FINAL_ERA_2017, FINAL_ERA_2018, FINAL_ERA_2019])

In [ ]:
FINAL_ERA

,최종방어율,120경기 방어율,예측최종방어율,잔여경기방어율,예측잔여경기방어율
LG,5.039348,5.320984,5.265632,3.631167,4.988874
SS,5.642839,5.691755,5.656922,5.398256,5.482758
HH,5.757865,5.857939,5.756932,5.257496,5.251897
LT,5.628850,5.584440,5.690406,5.850897,6.220234
NC,4.477844,4.663023,4.560663,3.551948,4.048861
HT,4.972571,5.140044,5.039196,4.135204,4.534960
WO,4.955741,4.811549,4.844981,5.676698,5.012141
SK,4.867347,4.812049,5.055729,5.143837,6.274132
KT,5.924588,5.849015,5.957534,6.302449,6.500126
OB,4.445250,4.392425,4.530029,4.709375,5.218052


**승률 예측하기**

In [ ]:
#연도별로 팀 별 경기당 승무패 데이터 불러오기
YEAR=[2016, 2017, 2018, 2019]
#(/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/)까지 경로를 수정하셔야 합니다.
for i in YEAR:  
  s1="""team_%d=pd.read_csv('/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_%d.csv')
team_%d=team_%d[['G_ID', 'T_ID', 'WLS']]
team_%d['WLS'].replace(['W','L','D'],[1,0,0], inplace=True)
team_%d['WLS']=team_%d['WLS'].astype('float64')
"""%(i, i, i, i, i, i, i)
  exec(s1)

In [ ]:
#연도별 데이터를 정제해 경기 진행에 따른 누적승률을 구할 수 있는 모델을 만듬
LAST_WIN_2016=[]
LAST_WIN_2017=[]
LAST_WIN_2018=[]
LAST_WIN_2019=[]

YEAR=[2016, 2017, 2018, 2019]
TEAM_LIST = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']
for i in YEAR:
  for j in TEAM_LIST:
    s1 = "%s_WIN_%d = pd.DataFrame(team_%d[team_%d['T_ID']==j])"%(j, i, i, i)
    exec(s1)

for i in YEAR:
  for j in TEAM_LIST:
    s1 = "%s_WIN_%d.reset_index(drop=True, inplace=True)"%(j, i)
    exec(s1)

for i in YEAR:
  for j in TEAM_LIST:
    s2 = "%s_WIN_%d['누적승리'] = %s_WIN_%d['WLS'].cumsum()"%(j,i, j, i)
    exec(s2)

for i in YEAR:
  for j in TEAM_LIST:
    s3 = "%s_WIN_%d['누적승률'] = %s_WIN_%d['누적승리']/(%s_WIN_%d.index+1)"%(j,i,j,i, j,i)
    exec(s3)

for i in YEAR:
  for j in TEAM_LIST:
    s11="LAST_WIN_%d.append(%s_WIN_%d['누적승률'][143])"%(i, j,i)
    exec(s11)

In [ ]:
#모델을 돌려 연도별 144경기 예측값 도출
TEAM_WIN = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']
PRED_WIN=[]
PRED_WIN_2016=[]
PRED_WIN_2017=[]
PRED_WIN_2018=[]
PRED_WIN_2019=[]
YEAR=[2016, 2017, 2018, 2019]

for i in YEAR:
  PRED_WIN=[]
  for j in TEAM_WIN:
    s10 = "data=%s_WIN_%d[['누적승률']]"%(j, i)
    exec(s10)
    data_raw = data.values.astype("float32")
    data_raw=np.array(data_raw)
    data_raw=data_raw.reshape(-1,1)
    scaler = MinMaxScaler(feature_range = (0, 1))
    dataset = scaler.fit_transform(data_raw)

    train=dataset

    window_size = 24
    train_X, train_Y = create_dataset(train, window_size)

    train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))

    # Fit the first model.
    model1 = fit_model(train_X, train_Y, window_size)

    pred=scaler.inverse_transform(model1.predict(train_X))

    PRED_WIN.append(pred[-1][0])

  s12 = "PRED_WIN_%d=PRED_WIN"%(i)
  exec(s12)

Epoch 1/50
95/95 - 0s - loss: 0.0025
Epoch 2/50
95/95 - 0s - loss: 0.0016
Epoch 3/50
95/95 - 0s - loss: 0.0016
Epoch 4/50
95/95 - 0s - loss: 0.0015
Epoch 5/50
95/95 - 0s - loss: 0.0016
Epoch 6/50
95/95 - 0s - loss: 0.0014
Epoch 7/50
95/95 - 0s - loss: 0.0014
Epoch 8/50
95/95 - 0s - loss: 0.0013
Epoch 9/50
95/95 - 0s - loss: 0.0014
Epoch 10/50
95/95 - 0s - loss: 0.0013
Epoch 11/50
95/95 - 0s - loss: 0.0013
Epoch 12/50
95/95 - 0s - loss: 0.0014
Epoch 13/50
95/95 - 0s - loss: 0.0014
Epoch 14/50
95/95 - 0s - loss: 0.0013
Epoch 15/50
95/95 - 0s - loss: 0.0013
Epoch 16/50
95/95 - 0s - loss: 0.0013
Epoch 17/50
95/95 - 0s - loss: 0.0016
Epoch 18/50
95/95 - 0s - loss: 0.0015
Epoch 19/50
95/95 - 0s - loss: 0.0012
Epoch 20/50
95/95 - 0s - loss: 0.0012
Epoch 21/50
95/95 - 0s - loss: 0.0013
Epoch 22/50
95/95 - 0s - loss: 0.0013
Epoch 23/50
95/95 - 0s - loss: 0.0013
Epoch 24/50
95/95 - 0s - loss: 0.0013
Epoch 25/50
95/95 - 0s - loss: 0.0012
Epoch 26/50
95/95 - 0s - loss: 0.0013
Epoch 27/50
95/95 - 0

In [ ]:
#최종 승률 예측값 산출

FINAL_WIN_2016=pd.DataFrame(data=[LAST_WIN_2016,PRED_WIN_2016], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_WIN_2016=FINAL_WIN_2016.T
FINAL_WIN_2017=pd.DataFrame(data=[LAST_WIN_2017,PRED_WIN_2017], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_WIN_2017=FINAL_WIN_2017.T
FINAL_WIN_2018=pd.DataFrame(data=[LAST_WIN_2018,PRED_WIN_2018], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_WIN_2018=FINAL_WIN_2018.T
FINAL_WIN_2019=pd.DataFrame(data=[LAST_WIN_2019,PRED_WIN_2019], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_WIN_2019=FINAL_WIN_2019.T

In [ ]:
#120경기 기존 값 도출
HALF_WIN_2016=[]
HALF_WIN_2017=[]
HALF_WIN_2018=[]
HALF_WIN_2019=[]

for i in YEAR:
  for j in TEAM_LIST:
    s11="HALF_WIN_%d.append(%s_WIN_%d['누적승률'][119])"%(i, j,i)
    exec(s11)

In [ ]:
#역추정하는 식
YEAR=[2016, 2017, 2018, 2019]
for i in YEAR:  
  s1="""FINAL_WIN_%d=pd.DataFrame(data=[LAST_WIN_%d,HALF_WIN_%d, PRED_WIN_%d], columns=['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB'])
FINAL_WIN_%d=FINAL_WIN_%d.T
FINAL_WIN_%d.columns=['최종승률', '120경기 승률', '예측최종승률']
FINAL_WIN_%d['잔여경기승률']=(FINAL_WIN_%d['최종승률']-(120/144)*FINAL_WIN_%d['120경기 승률'])*(144/24)
FINAL_WIN_%d['예측잔여경기승률']=(FINAL_WIN_%d['예측최종승률']-(120/144)*FINAL_WIN_%d['120경기 승률'])*(144/24)
"""%(i, i, i, i, i, i, i, i, i, i, i, i, i)
  exec(s1)

In [ ]:
FINAL_WIN=pd.concat([FINAL_WIN_2016, FINAL_WIN_2017, FINAL_WIN_2018, FINAL_WIN_2019])

In [ ]:
FINAL_WIN

,최종승률,120경기 승률,예측최종승률,잔여경기승률,예측잔여경기승률
LG,0.493056,0.475000,0.471186,0.583333,0.452114
SS,0.451389,0.441667,0.439166,0.500000,0.426662
HH,0.458333,0.450000,0.447859,0.500000,0.437153
LT,0.458333,0.450000,0.451028,0.500000,0.456165
NC,0.576389,0.583333,0.592546,0.541667,0.638611
HT,0.486111,0.491667,0.492866,0.458333,0.498861
WO,0.534722,0.566667,0.549702,0.375000,0.464879
SK,0.479167,0.483333,0.482659,0.458333,0.479290
KT,0.368056,0.383333,0.363267,0.291667,0.262938
OB,0.645833,0.641667,0.662251,0.666667,0.765170


# **2020년 예측 모델(중요)**

1. 타율

In [ ]:
#데이터 프레임임을 시계열로 넣을 수 있게 데이터 정제
def create_dataset(dataset, window_size):
  data_X, data_Y=[], []
  for i in range(len(dataset)-window_size-window_size-1):
    a= dataset[i:(i+window_size), 0]
    data_X.append(a)
    data_Y.append(dataset[i+window_size+window_size, 0])
  return(np.array(data_X), np.array(data_Y))

def fit_model(train_X, train_Y, window_size):
  model = Sequential()
    
  model.add(LSTM(4, 
                   input_shape=(1, window_size)))
  model.add(Dense(10))
  model.add(Dense(1))
  model.compile(loss = "mean_squared_error", optimizer = "adam")
  model.fit(train_X, train_Y, epochs = 50, batch_size = 1, verbose = 2)
  return(model)

In [ ]:
#(/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/)까지 경로를 수정하셔야 합니다.
bat=pd.read_csv('/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/teambat2020_시계열.csv')

In [ ]:
bat=bat[['GDAY_DS','T_ID','AB','HIT']]
TEAM_LIST = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']

for j in TEAM_LIST:
  s1 = "%s_AVG = pd.DataFrame(bat[bat['T_ID']==j])"%(j)
  exec(s1)


for j in TEAM_LIST:
  s2 = "%s_AVG['누적타석'] = %s_AVG['AB'].cumsum()"%(j, j)
  exec(s2)

for j in TEAM_LIST:
  s3 = "%s_AVG['누적안타'] = %s_AVG['HIT'].cumsum()"%(j,j)
  exec(s3)


for j in TEAM_LIST:
  s4 = "%s_AVG['누적타율'] = %s_AVG['누적안타']/%s_AVG['누적타석']"%(j, j, j)
  exec(s4)

for j in TEAM_LIST:
  s5 = "%s_AVG.reset_index(drop=True, inplace=True)"%(j)
  exec(s5)


In [ ]:
def pred(data, window): #잔여경기수
  data=data[['누적타율']]
  data_raw = data.values.astype("float32")
  data_raw=np.array(data_raw)
  data_raw=data_raw.reshape(-1,1)
  scaler = MinMaxScaler(feature_range = (0, 1))
  dataset = scaler.fit_transform(data_raw)

  train=dataset

  window_size = window #잔여경기 수
  train_X, train_Y = create_dataset(train, window_size)

  train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))

  # Fit the first model.
  model1 = fit_model(train_X, train_Y, window_size)

  pred=scaler.inverse_transform(model1.predict(train_X))
 
  return pred[-1]



In [ ]:
TEAM_LIST = {'LG':25, 'SS':26, 'HH':26, 'LT':28, 'NC':28, 'HT':29, 'WO':20, 'SK':24, 'KT':27, 'OB':25}
for i,j in zip(TEAM_LIST.keys(), TEAM_LIST.values()):
  s3 = """%s=pred(%s_AVG, %d)
REST_%s=(%s-((144-%d)/144)*%s_AVG['누적타율'][144-%d-1])*(144/%d)
"""%(i,i,j,i,i,j,i,j,j)
  exec(s3)

Epoch 1/50
68/68 - 0s - loss: 0.0476
Epoch 2/50
68/68 - 0s - loss: 0.0013
Epoch 3/50
68/68 - 0s - loss: 0.0013
Epoch 4/50
68/68 - 0s - loss: 0.0011
Epoch 5/50
68/68 - 0s - loss: 0.0012
Epoch 6/50
68/68 - 0s - loss: 0.0010
Epoch 7/50
68/68 - 0s - loss: 0.0011
Epoch 8/50
68/68 - 0s - loss: 9.5120e-04
Epoch 9/50
68/68 - 0s - loss: 9.1177e-04
Epoch 10/50
68/68 - 0s - loss: 8.3124e-04
Epoch 11/50
68/68 - 0s - loss: 7.7938e-04
Epoch 12/50
68/68 - 0s - loss: 8.4218e-04
Epoch 13/50
68/68 - 0s - loss: 8.1785e-04
Epoch 14/50
68/68 - 0s - loss: 8.8794e-04
Epoch 15/50
68/68 - 0s - loss: 7.1950e-04
Epoch 16/50
68/68 - 0s - loss: 7.5737e-04
Epoch 17/50
68/68 - 0s - loss: 7.4718e-04
Epoch 18/50
68/68 - 0s - loss: 6.7912e-04
Epoch 19/50
68/68 - 0s - loss: 7.9120e-04
Epoch 20/50
68/68 - 0s - loss: 6.6036e-04
Epoch 21/50
68/68 - 0s - loss: 6.5357e-04
Epoch 22/50
68/68 - 0s - loss: 6.6191e-04
Epoch 23/50
68/68 - 0s - loss: 6.7253e-04
Epoch 24/50
68/68 - 0s - loss: 6.2231e-04
Epoch 25/50
68/68 - 0s - loss

In [ ]:
rest_avg=[]
for i in TEAM_LIST.keys():
  s3 = "rest_avg.append(REST_%s[0])"%(i)
  exec(s3)

rest_avg=pd.DataFrame(rest_avg, index=TEAM_LIST.keys(), columns=['잔여경기예측타율'] )

In [ ]:
#결과
rest_avg

,잔여경기예측타율
LG,0.313374
SS,0.276330
HH,0.233434
LT,0.271669
NC,0.283232
HT,0.277559
WO,0.276936
SK,0.249663
KT,0.311416
OB,0.348003


In [ ]:
#저장 - 파일명(PRED_BAT_4)
#(/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/)까지 경로를 수정하셔야 합니다.
rest_avg.to_csv('/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/PRED_BAT_4.csv', index = False)

2. 방어율

In [ ]:
#(/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/)까지 경로를 수정하셔야 합니다.
pitcher=pd.read_csv('/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/teampit2020_시계열.csv')

In [ ]:
pitcher

,GDAY_DS,T_ID,INN2,ER
0,20200505OBLGDH0,OB,24.0,8
1,20200506OBLGDH0,OB,26.0,1
2,20200507OBLGDH0,OB,26.0,3
3,20200508KTOBDH0,OB,27.0,12
4,20200510KTOBDH0,OB,33.0,12
...,...,...,...,...
1177,20200923SSNCDH0,SS,24.0,8
1178,20200924SSOBDH0,SS,23.0,1
1179,20200925SSOBDH0,SS,27.0,3
1180,20200926SKSSDH0,SS,27.0,2


In [ ]:
pitcher=pitcher[['GDAY_DS', 'T_ID', 'INN2', 'ER']]
TEAM_LIST = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']

for j in TEAM_LIST:
  s1 = "%s_ERA = pd.DataFrame(pitcher[pitcher['T_ID']==j])"%(j)
  exec(s1)


for j in TEAM_LIST:
  s2 = "%s_ERA['누적이닝'] = %s_ERA['INN2'].cumsum()"%(j, j)
  exec(s2)

for j in TEAM_LIST:
  s3 = "%s_ERA['누적자책'] = %s_ERA['ER'].cumsum()"%(j,j)
  exec(s3)


for j in TEAM_LIST:
  s4 = "%s_ERA['누적방어율'] = %s_ERA['누적자책']*27/%s_ERA['누적이닝']"%(j, j, j)
  exec(s4)


for j in TEAM_LIST:
  s5 = "%s_ERA.reset_index(drop=True, inplace=True)"%(j)
  exec(s5)


In [ ]:
def pred_era(data, window): #잔여경기수
  data=data[['누적방어율']]
  data_raw = data.values.astype("float32")
  data_raw=np.array(data_raw)
  data_raw=data_raw.reshape(-1,1)
  scaler = MinMaxScaler(feature_range = (0, 1))
  dataset = scaler.fit_transform(data_raw)

  train=dataset

  window_size = window #잔여경기 수
  train_X, train_Y = create_dataset(train, window_size)

  train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))

  # Fit the first model.
  model1 = fit_model(train_X, train_Y, window_size)

  pred=scaler.inverse_transform(model1.predict(train_X))
 
  return pred[-1]

In [ ]:
TEAM_LIST = {'LG':25, 'SS':26, 'HH':26, 'LT':28, 'NC':28, 'HT':29, 'WO':20, 'SK':24, 'KT':27, 'OB':25}
for i,j in zip(TEAM_LIST.keys(), TEAM_LIST.values()):
  s3 = """%s=pred_era(%s_ERA, %d)
REST_ERA_%s=(%s-((144-%d)/144)*%s_ERA['누적방어율'][144-%d-1])*(144/%d)
"""%(i,i,j,i,i,j,i,j,j)
  exec(s3)


Epoch 1/50
68/68 - 0s - loss: 0.0162
Epoch 2/50
68/68 - 0s - loss: 0.0027
Epoch 3/50
68/68 - 0s - loss: 0.0025
Epoch 4/50
68/68 - 0s - loss: 0.0026
Epoch 5/50
68/68 - 0s - loss: 0.0023
Epoch 6/50
68/68 - 0s - loss: 0.0020
Epoch 7/50
68/68 - 0s - loss: 0.0018
Epoch 8/50
68/68 - 0s - loss: 0.0018
Epoch 9/50
68/68 - 0s - loss: 0.0015
Epoch 10/50
68/68 - 0s - loss: 0.0017
Epoch 11/50
68/68 - 0s - loss: 0.0014
Epoch 12/50
68/68 - 0s - loss: 0.0012
Epoch 13/50
68/68 - 0s - loss: 0.0011
Epoch 14/50
68/68 - 0s - loss: 0.0012
Epoch 15/50
68/68 - 0s - loss: 9.1642e-04
Epoch 16/50
68/68 - 0s - loss: 9.1744e-04
Epoch 17/50
68/68 - 0s - loss: 9.9116e-04
Epoch 18/50
68/68 - 0s - loss: 8.0255e-04
Epoch 19/50
68/68 - 0s - loss: 6.9339e-04
Epoch 20/50
68/68 - 0s - loss: 6.5683e-04
Epoch 21/50
68/68 - 0s - loss: 8.2726e-04
Epoch 22/50
68/68 - 0s - loss: 5.5302e-04
Epoch 23/50
68/68 - 0s - loss: 5.2871e-04
Epoch 24/50
68/68 - 0s - loss: 5.1944e-04
Epoch 25/50
68/68 - 0s - loss: 5.7083e-04
Epoch 26/50
68/

In [ ]:
rest_era=[]
for i in TEAM_LIST.keys():
  s3 = "rest_era.append(REST_ERA_%s[0])"%(i)
  exec(s3)

rest_era=pd.DataFrame(rest_era, index=TEAM_LIST.keys(), columns=['잔여경기예측방어율'] )

In [ ]:
#예측결과
rest_era

,잔여경기예측방어율
LG,4.646109
SS,3.622887
HH,5.822363
LT,4.574877
NC,5.289544
HT,4.944839
WO,4.685803
SK,5.499825
KT,4.393822
OB,4.645090


In [ ]:
#저장 - 파일명(PRED_PIT_4)
#(/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/)까지 경로를 수정하셔야 합니다.
rest_era.to_csv('/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/PRED_PIT_4.csv', index = False)

3. 승률 예측

In [64]:
#(/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/)까지 경로를 수정하셔야 합니다.
win=pd.read_csv('/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/wls2020_시계열.csv')

In [69]:
TEAM_LIST = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']

for j in TEAM_LIST:
  s1 = "%s_WIN = pd.DataFrame(win[win['T_ID']==j])"%(j)
  exec(s1)

for j in TEAM_LIST:
  s5 = "%s_WIN.reset_index(drop=True, inplace=True)"%(j)
  exec(s5)


In [70]:
def pred_win(data, window): #잔여경기수
  data=data[['누적승률']]
  data_raw = data.values.astype("float32")
  data_raw=np.array(data_raw)
  data_raw=data_raw.reshape(-1,1)
  scaler = MinMaxScaler(feature_range = (0, 1))
  dataset = scaler.fit_transform(data_raw)

  train=dataset

  window_size = window #잔여경기 수
  train_X, train_Y = create_dataset(train, window_size)

  train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))

  # Fit the first model.
  model1 = fit_model(train_X, train_Y, window_size)

  pred=scaler.inverse_transform(model1.predict(train_X))
 
  return pred[-1]

In [71]:
TEAM_LIST = {'LG':25, 'SS':26, 'HH':26, 'LT':28, 'NC':28, 'HT':29, 'WO':20, 'SK':24, 'KT':27, 'OB':25}
for i,j in zip(TEAM_LIST.keys(), TEAM_LIST.values()):
  s3 = """%s=pred_win(%s_WIN, %d)
REST_WIN_%s=(%s-((144-%d)/144)*%s_WIN['누적승률'][144-%d-1])*(144/%d)
"""%(i,i,j,i,i,j,i,j,j)
  exec(s3)


Epoch 1/50
68/68 - 0s - loss: 0.0066
Epoch 2/50
68/68 - 0s - loss: 0.0027
Epoch 3/50
68/68 - 0s - loss: 0.0023
Epoch 4/50
68/68 - 0s - loss: 0.0020
Epoch 5/50
68/68 - 0s - loss: 0.0015
Epoch 6/50
68/68 - 0s - loss: 0.0013
Epoch 7/50
68/68 - 0s - loss: 0.0010
Epoch 8/50
68/68 - 0s - loss: 8.8004e-04
Epoch 9/50
68/68 - 0s - loss: 6.2413e-04
Epoch 10/50
68/68 - 0s - loss: 5.2774e-04
Epoch 11/50
68/68 - 0s - loss: 4.1199e-04
Epoch 12/50
68/68 - 0s - loss: 4.3636e-04
Epoch 13/50
68/68 - 0s - loss: 3.0447e-04
Epoch 14/50
68/68 - 0s - loss: 3.0846e-04
Epoch 15/50
68/68 - 0s - loss: 2.7306e-04
Epoch 16/50
68/68 - 0s - loss: 2.4731e-04
Epoch 17/50
68/68 - 0s - loss: 2.3283e-04
Epoch 18/50
68/68 - 0s - loss: 2.1267e-04
Epoch 19/50
68/68 - 0s - loss: 1.9432e-04
Epoch 20/50
68/68 - 0s - loss: 1.9127e-04
Epoch 21/50
68/68 - 0s - loss: 1.9203e-04
Epoch 22/50
68/68 - 0s - loss: 2.0663e-04
Epoch 23/50
68/68 - 0s - loss: 2.2721e-04
Epoch 24/50
68/68 - 0s - loss: 1.9069e-04
Epoch 25/50
68/68 - 0s - loss

In [72]:
rest_win=[]
for i in TEAM_LIST.keys():
  s3 = "rest_win.append(REST_WIN_%s[0])"%(i)
  exec(s3)

rest_win=pd.DataFrame(rest_win, index=TEAM_LIST.keys(), columns=['잔여경기예측승률'] )

In [ ]:
#저장 - 파일명(PRED_WIN_3)
#(/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/)까지 경로를 수정하셔야 합니다.
rest_win.to_csv('/content/drive/Shared drives/TNT/프로젝트/2020년 2학기/빅콘/코드/0926 최종/박준영이 쓴 파일/PRED_WIN_3.csv', index = False)

In [ ]:
#최종 예측 결과
FINAL=pd.concat([rest_avg, rest_era, rest_win], axis=1)
FINAL

,잔여경기예측타율,잔여경기예측방어율,잔여경기예측승률
LG,0.313374,4.646642,0.604515
SS,0.276330,3.650081,0.514909
HH,0.233434,6.100301,0.208519
LT,0.271669,4.288703,0.563368
NC,0.283232,5.382896,0.492934
HT,0.277559,4.878848,0.603814
WO,0.276936,4.663851,0.670433
SK,0.249663,4.427788,0.277516
KT,0.311416,4.538348,0.529114
OB,0.348003,4.736650,0.637590
